In [6]:
import dotenv
import duckdb
import pathlib

from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame

from tqdm.notebook import tqdm

# Identify the Stocks and Years

In [7]:
filename = pathlib.Path('sp500.txt')

with filename.open() as fp:
    stocks = [line.strip() for line in fp.readlines()]

years = [
    2022,
    2023,
]

# Fetch and Save the Historical Stock Data

In [4]:
# load secrets
env = dotenv.dotenv_values('alpaca.env')

# create client
client = StockHistoricalDataClient(
    api_key=env['API_KEY'],
    secret_key=env['SECRET_KEY'],
)

# process each year
for year in tqdm(years):
    req_params = StockBarsRequest(
        symbol_or_symbols=stocks,
        timeframe=TimeFrame.Day,
        start=f'{year}-01-01',
        end=f'{year}-12-31',
    )

    # download the history
    resp = client.get_stock_bars(req_params)

    # convert to a dataframe with stock symbol and date columns
    df = resp.df.rename_axis(['stock', 'date']).reset_index()

    # remove the timestamp from the date
    df['date'] = df['date'].dt.date

    # use the target symbol for the output filename
    out = f'{filename.stem}_{year}.db'

    # save it to disk as a database
    with duckdb.connect(out) as con:
        con.sql("CREATE TABLE stocks AS SELECT * FROM df")

  0%|          | 0/2 [00:00<?, ?it/s]